<a href="https://colab.research.google.com/github/Many98/GLM_R2Py/blob/main/Python/01ZLMA_ex03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01ZLMA - Exercise 03
Exercise 03 of the course 01ZLMA. 

## Contents

* Statistical Inference
 ---
* Testing
 ---




#  Necessary theory recap from Lecture 04

Under the conditions of regularity holds

1.  $ \ U(\beta) \sim N_{p}(0,I(\beta)) \Rightarrow  I^{-\frac{1}{2}}(\beta)\, U(\beta) {\stackrel{D}{\longrightarrow}} N_{p}(0, 1)$
2. $ U(\beta)I^{-1}(\beta)U(\beta)\sim \chi^{2}(p) \Rightarrow U(\beta)^T I^{-1}(\beta)U(\beta)  {\stackrel{D}{\longrightarrow}} \chi^{2}(p)$
3. Consistency of $\hat{\beta}$ and Wald statistics: \\
 $\hat{\beta}\sim N_{p}(\beta,I^{-1}(\beta)) \Rightarrow
(\hat{\beta}-\beta)^T I(\beta)(\hat{\beta}-\beta) {\stackrel{D}{\longrightarrow}} \chi^{2}(p)$



Saturated and null model

* Null model: $\mu_i = \mu, \forall i \in \{1, \ldots , n\}$ \\
The Null Model assumes one parameter for all of the data points, which means you only estimate 1 parameter. 
* Saturated model: $Y_i = \hat{\mu_i}, \forall i \in \{1, \ldots , n\}$ \\
The Saturated Model is a model that assumes each data point has its own parameters, which means you have n parameters to estimate.
* Proposed Model:  model, where you try to explain your data points with $p$ parameters + an intercept term, so you have p+1 parameters, where $1 \leq p \leq n$.

Questions:
* What is the difference between null and saturated model?
* Which model has greater log-likelihoood value?
* Which model has the highest log-likelihood value?
* What can you say about asymptotic distributions of $\hat{\beta}$ and $U(\hat{\beta})$ for saturated model?

## Let's code 

In [1]:
import numpy as np
import scipy
from scipy import stats

import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn

import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.api import abline_plot

import pandas as pd

Use Example 2 from the last Exercise 02

In [2]:
n  = 20 # n observations
m  = 2 # m parameters to estimate
X1 = np.ones((n*m,))  # Intercept
X2 = np.array([i for i in range(1, n+1)] * m) # Regressors
X = np.vstack([X1, np.log(X2)]).T # design matrix
beta = np.array([0.9, 1.3]) # Regression coefficients
lamdas = np.exp(X @ beta) # Means
Y = np.random.poisson(lamdas, n*m) # Response variable with Poisson distribution

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   40
Model:                            GLM   Df Residuals:                       38
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -118.52
Date:                Sat, 03 Sep 2022   Deviance:                       19.573
Time:                        18:00:52   Pearson chi2:                     19.8
No. Iterations:                     5   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8079      0.133      6.076      0.0

In [3]:
# to create models we can use standard api or formula (r-like) api in statsmodels
# formula api requires dataset (pandas) and formula
d = pd.DataFrame(data={'Y': Y, 'X1': X1, 'X2':X2})
#model = smf.glm(formula='Y~np.log(X2)', data=d, family=sm.families.Poisson()).fit()

# standard api requires specifying endog (response) and exog (explanatory) design matrices
model = sm.GLM(endog=Y, exog=X, family=sm.families.Poisson()).fit()
print(model.summary())

beta_e = model.params; print(f'estimated params are:{beta_e}')
y_hat = model.predict(); print(f'fitted values are:{y_hat}')

fig, ax = plt.subplots()
ax.scatter(X2, Y, color='red', marker='o')
ax.plot(np.unique(y_hat), color='blue')
ax.set_title('Poisson model')
ax.set_xlabel('Year Quoter')
ax.set_ylabel('Number of cases')
plt.show()

Repetition using custom function:

In [4]:
# function to calcualate weights W
def calc_W_inv(X, beta):
    return np.diag(np.exp(X @ beta))

In [5]:
# function to calcualate weights Z
def calc_Z(X,Y,beta):
    return X@beta + (Y - np.exp(X@beta)) / np.exp(X@beta)

In [6]:
# IWLS for example 2

def IWLS(X,Y,beta_init,maxiter,epsilon):
    res = {'FM': None, 'SV': None, 'betas': None}
    # Fisher-scoring algorithm
    i = 1     # first iteration

    beta_i = beta_init
    
    while i <= maxiter:
        W = calc_W_inv(X,beta_i)
        Z = calc_Z(X,Y,beta_i)
        beta_pred = beta_i
        beta_i = np.linalg.solve(X.T@W@X, X.T@W@Z)
        diff = np.max(np.abs(beta_i - beta_pred))
        if diff < epsilon:
            break
        W = calc_W_inv(X, beta_i)
        Z = calc_Z(X, Y, beta_i)

        res['SV'] = X.T@W@Z
        res['FM'] = X.T@W@X
        res['betas'] = np.linalg.solve(X.T@W@X, X.T@W@Z)
        i += 1
    return res

In [7]:
# Estimation of betas
result1 = IWLS(X,Y,np.ones(2),100,10^(-6))
print(f'Estimation of parameters: {result1["betas"]}')      # Estimation of parameters
print(f'Estimated Fisher information matrix: {result1["FM"]}')        # Estimated Fisher information matrix
print(f'Estimated covariance matrix: {np.linalg.inv(result1["FM"])}')  # Estimated covariance matrix  = Inverse of estimated Fisher information matrix


Estimation of parameters: [0.80787794 1.3162756 ]
Estimated Fisher information matrix: [[ 2115.          5475.27622876]
 [ 5475.27622876 14563.78984362]]
Estimated covariance matrix: [[ 0.01767953 -0.00664664]
 [-0.00664664  0.00256748]]


Comparison of our custom solution with the built in glm function:

In [8]:
print(model.summary())

# the unscaled (dispersion = 1) estimated covariance matrix of the estimated coefficients.
FIM1 = model.cov_params()
print(f'estimated covariance matrix {FIM1}')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   40
Model:                            GLM   Df Residuals:                       38
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -118.52
Date:                Sat, 03 Sep 2022   Deviance:                       19.573
Time:                        18:00:53   Pearson chi2:                     19.8
No. Iterations:                     5   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8079      0.133      6.076      0.0

In [9]:
# to find out what params has `model` object
for attr in dir(model):
    if not attr.startswith('_'):
        print(attr)

aic
bic
bic_deviance
bic_llf
bse
conf_int
converged
cov_kwds
cov_params
cov_type
deviance
df_model
df_resid
f_test
family
fit_history
fittedvalues
get_hat_matrix_diag
get_influence
get_prediction
info_criteria
initialize
k_constant
llf
llf_scaled
llnull
load
method
mle_settings
model
mu
nobs
normalized_cov_params
null
null_deviance
params
pearson_chi2
plot_added_variable
plot_ceres_residuals
plot_partial_residuals
predict
pseudo_rsquared
pvalues
remove_data
resid_anscombe
resid_anscombe_scaled
resid_anscombe_unscaled
resid_deviance
resid_pearson
resid_response
resid_working
save
scale
summary
summary2
t_test
t_test_pairwise
tvalues
use_t
wald_test
wald_test_terms


Asymptotics:

* $ (\hat{\beta} - \beta) \sim N_{p}(0, I^{-1}(\beta))$ 
* Estimated Fisher information matrix  $\hat{I}(\hat{\beta}) = (X^T \hat{W} X)$  matrix.
*  Estimated covariance matrix $\hat{V} (\hat{\beta}) = (X^T \hat{W} X)^{-1}$ 

In [10]:
n = 10
repet = 50
n_observ = np.array([1,2,5,10,100, 500])
betas_hat = np.zeros((6, repet, 2))

for _, i in enumerate(n_observ):
    for j in range(repet):
        X1 = np.ones((n*i,))
        X2 = np.array([i for i in range(1, n+1)]*i)
        X  = np.vstack([X1, np.log(X2)]).T
        beta = np.array([0.9, 1.3]) # Regression coefficients
        lamdas = np.exp(X @ beta) # Means
        Y = np.random.poisson(lamdas, n*i)
        betas_hat[_, j] = sm.GLM(endog=Y, exog=X, family=sm.families.Poisson()).fit().params


In [11]:

for i in range(len(n_observ)):
    print(f"Number of observations: {n_observ[i]*n}")
    print(np.cov((betas_hat[i] - beta).T))
    print(np.mean(betas_hat[i] - beta))

Number of observations: 10
[[ 0.06444145 -0.03109597]
 [-0.03109597  0.01611895]]
0.008330369352331961
Number of observations: 20
[[ 0.04380673 -0.02140931]
 [-0.02140931  0.01083468]]
0.005453865374317191
Number of observations: 50
[[ 0.01484336 -0.00770914]
 [-0.00770914  0.00418468]]
-0.0017041343074835657
Number of observations: 100
[[ 0.0086264  -0.00425775]
 [-0.00425775  0.00218247]]
-0.0025229516345396143
Number of observations: 1000
[[ 0.00146265 -0.0007335 ]
 [-0.0007335   0.00037602]]
0.0011531601930317614
Number of observations: 5000
[[ 2.05562714e-04 -1.06502108e-04]
 [-1.06502108e-04  5.75415090e-05]]
-0.000398511838356842


## Hypothesis testing

Use the model from the beginning again.

In [12]:
n  = 20
m  = 2

X1 = np.ones((n*m,))
X2 = np.array([i for i in range(1, n+1)]*m)
X  = np.vstack([X1, np.log(X2)]).T
beta = np.array([0.9, 1.3]) # Regression coefficients
lamdas = np.exp(X @ beta) # Means
Y = np.random.poisson(lamdas, n*m)



In [13]:
model = sm.GLM(endog=Y, exog=X, family=sm.families.Poisson()).fit()
print(model.summary())

# the unscaled (dispersion = 1) estimated covariance matrix of the estimated coefficients.
FIM1 = model.cov_params()
print(f'estimated covariance matrix {FIM1}')


                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   40
Model:                            GLM   Df Residuals:                       38
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -133.53
Date:                Sat, 03 Sep 2022   Deviance:                       49.362
Time:                        18:00:54   Pearson chi2:                     48.5
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7312      0.133      5.491      0.0

Calculation of Z value
 $$Z_i = \frac{\hat{\beta_i}}{(I^{-1}(\hat{\beta_i}))_{ii}}$$

In [14]:
# Testing statistics from summary table
model.summary()

 

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   40
Model:                            GLM   Df Residuals:                       38
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -133.53
Date:                Sat, 03 Sep 2022   Deviance:                       49.362
Time:                        18:00:54   Pearson chi2:                     48.5
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7312      0.133      5.491      0.000       0.470       0.992
x1             1.3607      0.051     26.881      0.000       1.261       1.460
==============================================================================
"""

In [15]:
# By definition

z_stat = model.params / np.sqrt(np.diag(model.cov_params()))
print(z_stat)
z_stat == model.tvalues

[ 5.49104794 26.88102202]


array([ True,  True])

In [16]:
# p-values of the test
p_val = 2*scipy.stats.norm.sf(z_stat, loc=0, scale=1)
print(f'pval: {p_val}')
p_val == model.pvalues

pval: [3.99555882e-008 3.66111009e-159]


array([ True,  True])

In [17]:
### 100(1-alpha) confidence interval
alpha = 0.05
u = scipy.stats.norm.ppf(1-alpha/2,0,1)
CI_LB = model.params[1] - u * np.sqrt(np.diag(model.cov_params())[1])
CI_UB = model.params[1] + u * np.sqrt(np.diag(model.cov_params())[1])

print(f"2.5% CI = {CI_LB},ESTIM = {model.params[1]}, 97.5% CI = {CI_UB}")


# built in function
print(model.conf_int())

2.5% CI = 1.2614738044805862,ESTIM = 1.360684809928891, 97.5% CI = 1.459895815377196
[[0.47022347 0.9922281 ]
 [1.2614738  1.45989582]]


Question:

* Compare hypothesis testing in LM vs. GLM

# Deviance

Deviance is a measure of goodness of fit of a GLM. 


Log-likelihood of the saturated model is the highest possible one with given data, $\tilde{\mu}_i = y_i$ and $\tilde{\theta_i} = \theta(y) = (b')^{-1}(y_i)$.
$$l(\tilde{\mu},\phi;y)=\sum_{i=1}^{n}\frac{y_{i}\tilde{\theta}_{i}-b(\tilde{\theta}_{i})}{a_{i}(\phi)}+\sum_{i=1}^{n}c(y_i,\phi)$$

Scale deviance statistics:
$${S(y,\hat{\mu},\phi)}=2\left[l(\tilde{\mu},\phi;y)-l(\hat{\mu},\phi;y)\right]
=2\sum_{i=1}^{n}\frac{y_{i}(\tilde{\theta}_{i}-\hat{\theta}_{i})
-\left(b(\tilde{\theta}_{i})-b(\hat{\theta}_{i})\right)}{a_{i}(\phi)}.
$$

Deviance:
Let $a_{i}(\phi)=a_{i}\phi$, then
$$S(y,\hat{\mu},\phi)=\frac{D(y,\hat{\mu})}{\phi},
$$
and
$$
D(y,\hat{\mu})=2\sum_{i=1}^{n}\frac{y_{i}(\tilde{\theta}_{i}-\hat{\theta}_{i})
-\left(b(\tilde{\theta}_{i})-b(\hat{\theta}_{i})\right)}{a_{i}}
$$

### Comparison of two models

Assume model $D_0$ with $p_0$ paramters and its sub-model $D_1$ with $p_1$ parameters, then
$$ \frac{1}{\phi} (D_0 - D_1) \sim \chi(p_0 - p_1) $$

Question:
* Can we take deviance as a measure of the model quality?
* Can we use deviance as a measure of the saturated model quality?
* Complete the sentence: Compare two GLMs with deviance is like compare two LMs with ...

In [18]:
# Add random variable to the previous model 
Z = scipy.stats.uniform.rvs(loc=0, scale=1, size=n*m)
model_0 = sm.GLM(endog=Y, exog=np.hstack([X, Z[:, None]]), family=sm.families.Poisson()).fit()
model_0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   40
Model:                            GLM   Df Residuals:                       37
Model Family:                 Poisson   Df Model:                            2
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -133.09
Date:                Sat, 03 Sep 2022   Deviance:                       48.495
Time:                        18:00:54   Pearson chi2:                     47.7
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7069      0.135      5.217      0.000       0.441       0.972
x1             1.3579      0.051     26.833      0.000       1.259       1.457
x2             0.0646      0.069      0.931      0.352      -0.071       0.201
==============================================================================
"""

In [39]:
# Proposed model
m1 = sm.GLM(endog=Y, exog=X, family=sm.families.Poisson())
model_1 = m1.fit()
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   40
Model:                            GLM   Df Residuals:                       38
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -133.53
Date:                Sat, 03 Sep 2022   Deviance:                       49.362
Time:                        18:52:13   Pearson chi2:                     48.5
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7312      0.133      5.491      0.000       0.470       0.992
x1             1.3607      0.051     26.881      0.000       1.261       1.460
==============================================================================
"""

In [20]:
# Null model

model_n = sm.GLM(endog=Y, exog=X[:, 0], family=sm.families.Poisson()).fit()
model_n.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   40
Model:                            GLM   Df Residuals:                       39
Model Family:                 Poisson   Df Model:                            0
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -701.15
Date:                Sat, 03 Sep 2022   Deviance:                       1184.6
Time:                        18:00:54   Pearson chi2:                 1.07e+03
No. Iterations:                     4   Pseudo R-squ. (CS):         -5.684e-14
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0064      0.021    187.833      0.000       3.965       4.048
==============================================================================
"""

In [21]:
# Saturated model CANNOT BY OBTAINED BY STATSMODELS BY DEFAULT 'CAUSE THEY PREVENT ZERO DIVISION

# BUT MY WORKAROUND WHICH PROBABLY WILL NOT WORK IN COLAB AS WE DO NOT HAVE ACCESS TO MODULES IS
# TO COMMENT OUT ROWS 1227-1229 IN MOST RECENT VERSION 
# (SEE https://github.com/statsmodels/statsmodels/blob/main/statsmodels/genmod/generalized_linear_model.py)

I = np.diag(np.ones((m*n,)))


model_s = sm.GLM(endog=Y, exog=I, family=sm.families.Poisson()).fit()
print(model_s.summary())
print(f'Residual deviance is: {model_s.deviance}')



                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   40
Model:                            GLM   Df Residuals:                        0
Model Family:                 Poisson   Df Model:                           39
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -108.85
Date:                Sat, 03 Sep 2022   Deviance:                   7.6383e-14
Time:                        18:00:54   Pearson chi2:                 1.19e-27
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -3.644e-17      1.000  -3.64e-17      1.0

/media/emanuel/data/miniconda3/lib/python3.9/site-packages/statsmodels/regression/_tools.py:121: RuntimeWarning: divide by zero encountered in double_scalars
  scale = np.dot(wresid, wresid) / df_resid


For Poisson model:
$$D = 2 \sum_{i=1}^n y_i log( \frac{y_i}{\hat{\mu_i}})$$

In [22]:
mu_est_0 = model_0.predict()
mu_est_1 = model_1.predict()

Dev_0 = 2*np.sum(Y*np.log(Y/mu_est_0))
print(Dev_0)
Dev_1 = 2*np.sum(Y*np.log(Y/mu_est_1))
print(Dev_1)


48.494759469850976
49.36227563895237


## Anova testing 
from anova.glm?

The table will optionally contain test statistics (and P values) comparing the reduction in deviance for the row to the residuals. For models with known dispersion (e.g., binomial and Poisson fits) the chi-squared test is most appropriate, and for those with dispersion estimated by moments (e.g., gaussian, quasibinomial and quasipoisson fits) the F test is most appropriate. 

Mallows' Cp statistic is the residual deviance plus twice the estimate of $sigma^2$ times the residual degrees of freedom, which is closely related to AIC (and a multiple of it if the dispersion is known). You can also choose "LRT" and "Rao" for likelihood ratio tests and Rao's efficient score test. The former is synonymous with "Chisq" (although both have an asymptotic chi-square distribution). 

In [23]:
anova(model_1)
anova(model_1, test = "Cp")
anova(model_1, test = "Chisq")

anova(model_1, model_0, test = "Rao")
anova(model_1, model_0, test = "LRT")   
  

NameError: name 'anova' is not defined

In [ ]:
# p-value of deviance tst
# H0: model fit data
p_dev <- pchisq(model_1$deviance, model_1$df.residual, ncp=0, lower.tail = FALSE)
p_dev

# critical value
C_val <- qchisq(0.05, model_1$df.residual, ncp=0, lower.tail = FALSE)
C_val

#summary(model_1)
#pchisq(1168 - 44, df=(39-38))

anova(model_1,model_s, test = "LRT")   # saturated vs. final model



In [24]:
model_1.nobs

40

In [ ]:
https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/anova.glm

class anova():
    def _call_(*models, test='chisq'):
        if len(models) == 1:
            """Testing quality of model"""
            res = _fork(*models, test)
        elif len(models) > 1:
            """Comparing models"""
        else:
            raise Exception('None model selected')
    def __str__():
        pass
    def _warnings():
        pass
    def _fork(*models, test):
        if test.lower() in ('chisq'.lower(), 'lrt'):
            res = _chisq_()
        elif test.lower() == 'rao':
            res = _rao_()
        elif test.lower() == 'cp'
            res = _cp_()
        else:
            raise Exception(f'Not such test implemented: {test}')
        return res
    def _score_():
        pass
    def _fisher_():
        pass
    def _F_(*models):
        """Performs deviance F-test most appropriate when scale param (phi) is not known"""
        phi_hat  = models[-1].deviance / (models[-1].nobs - models[-1].df_resid)
        f_stat = (models[0].deviance - models[-1].deviance) / ((models[-1].df_resid - models[0].df_resid)* phi_hat)
        p_val = scipy.stats.f.sf(f_stat, dfn=models[-1].df_resid - models[0].df_resid, 
                                 dfd=models[-1].nobs - models[-1].df_resid)
        return {'F_stat': f_stat, 'p_val': p_val}
    def _chisq_(*models):
        """Performs deviance LRT test leading to Chisq test statistic (known dispersion param)
            Note that if disperzion param is not known then deviance LRT test leads to F test and
            performing Chisq test is inappropriate
        """
        chi2_stat = (models[0].deviance - models[-1].deviance) / models[0].scale
        p_val = scipy.stats.chi2.sf(chi2_stat, df=models[-1].df_resid - models[0].df_resid)
        print(f'Estimated F statistic is: {f_stat} \n'
              f'P-value is: {p_val} \n'
              )
    def _rao_():
        raise Exception('Currently not implemented')
    def _cp_():
        raise Exception('Currently not implemented')

## Rao statistics

In [111]:
######## Rao score statistics

score = model_1.resid_response / model_1.model.family.variance(model_1.predict())
rao = score.T @  np.diag(model_1.model.family.variance(model_1.predict())) @ score
print(f'rao score statistic: {rao}')

# p-hodnota testu adekvatnosti modelu (pomoci Raovy statistiky)
# H0: model dobre popisuje data

print(f'p-val of rao test: {scipy.stats.chi2.sf(rao, df=model_1.df_resid)}')

######  pomoci saturovaneho modelu
model_1.score_test()
#anova(model_1,model_s, test = "Rao")

rao score statistic: 48.46844903288136
p-val of rao test: 0.11891923476215518


AttributeError: 'GLMResults' object has no attribute 'score_test'

In [88]:
model_1.model.family.variance(model_1.predict())

array([  2.07762577,   5.33548946,   9.26358238,  13.7019131 ,
        18.56287292,  23.78953264,  29.34130257,  35.18747886,
        41.3038573 ,  47.67076638,  54.27183825,  61.09319705,
        68.12289921,  75.35053364,  82.76692784,  90.36392651,
        98.1342213 , 106.07121755, 114.16892832, 122.42188896,
         2.07762577,   5.33548946,   9.26358238,  13.7019131 ,
        18.56287292,  23.78953264,  29.34130257,  35.18747886,
        41.3038573 ,  47.67076638,  54.27183825,  61.09319705,
        68.12289921,  75.35053364,  82.76692784,  90.36392651,
        98.1342213 , 106.07121755, 114.16892832, 122.42188896])

In [80]:
Y-mu_est_1

array([ -1.07762577,  -1.33548946,   1.73641762,   4.2980869 ,
        -6.56287292,   0.21046736,  -1.34130257,  -6.18747886,
         0.6961427 ,  -3.67076638,  12.72816175,  -1.09319705,
         7.87710079,   2.64946636,  16.23307216,   8.63607349,
        -6.1342213 ,   1.92878245,  -0.16892832,  15.57811104,
         0.92237423,   2.66451054,  -0.26358238,   1.2980869 ,
        -4.56287292,   2.21046736,  -2.34130257,  -1.18747886,
        -0.3038573 ,   8.32923362,  18.72816175, -11.09319705,
       -17.12289921, -25.35053364,  10.23307216, -14.36392651,
        -6.1342213 ,   6.92878245, -11.16892832,  -2.42188896])

In [85]:
model_1.resid_response

array([ -1.07762577,  -1.33548946,   1.73641762,   4.2980869 ,
        -6.56287292,   0.21046736,  -1.34130257,  -6.18747886,
         0.6961427 ,  -3.67076638,  12.72816175,  -1.09319705,
         7.87710079,   2.64946636,  16.23307216,   8.63607349,
        -6.1342213 ,   1.92878245,  -0.16892832,  15.57811104,
         0.92237423,   2.66451054,  -0.26358238,   1.2980869 ,
        -4.56287292,   2.21046736,  -2.34130257,  -1.18747886,
        -0.3038573 ,   8.32923362,  18.72816175, -11.09319705,
       -17.12289921, -25.35053364,  10.23307216, -14.36392651,
        -6.1342213 ,   6.92878245, -11.16892832,  -2.42188896])

In [82]:
model_1.resid_pearson @ model_1.resid_pearson.T 

48.46844903288135

In [109]:
def _F_(*models, estim_scale=False):
        """Performs deviance F-test most appropriate when scale param (phi) is not known"""
        phi_hat = models[0].scale
        if estim_scale:
            phi_hat  = models[-1].deviance / (models[-1].nobs - models[-1].df_model)
    
        f_stat = (models[0].deviance - models[-1].deviance) / ((models[-1].df_model - models[0].df_model)* phi_hat)
        p_val = scipy.stats.f.sf(f_stat, dfn=models[-1].df_model - models[0].df_model, 
                                 dfd=models[-1].nobs - models[-1].df_model)
        print(f'Estimated F statistic is: {f_stat} \n'
              f'P-value is: {p_val} \n'
              )
def _chisq_(*models):
    """Performs deviance LRT test leading to Chisq test statistic (known dispersion param)
        Note that if disperzion param is not known then deviance LRT test leads to F test and
        performing Chisq test is inappropriate
    """
    chi2_stat = (models[0].deviance - models[-1].deviance) / models[0].scale
    p_val = scipy.stats.chi2.sf(chi2_stat, df=models[-1].df_model - models[0].df_model)
    print(f'Estimated Chi2 statistic is: {chi2_stat} \n'
          f'P-value is: {p_val} \n'
          )

def _rao_(*models):
    scores = [i.resid_response / i.model.family.variance(i.predict()) for i in models]
    raos = [i.T  @ np.diag(j.model.family.variance(j.predict())) @ i for i, j in zip(scores, models)]
    
    chi2_stat = (raos[0] - raos[-1]) / models[0].scale
    p_val = scipy.stats.chi2.sf(chi2_stat, df=models[-1].df_model - models[0].df_model)
    print(f'Estimated Rao statistic is: {chi2_stat} \n'
          f'P-value is: {p_val} \n'
          )

In [110]:
_rao_(model_1, model_0)

Estimated Rao statistic is: 0.7584077948718644 
P-value is: 0.6161727410972954 



In [79]:
R = np.array(([1,1]))
model_1.wald_test(R, scalar=True)
#model_0.deviance


<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=626.2068650098701, p-value=3.340391444300058e-138, df_denom=1>

In [76]:
_F_(model_n, model_1)

Estimated F statistic is: 1135.2536376115138 
P-value is: 1.8879825021417397e-30 



# Your turn:
1. Generate data with followings parameters
 * $Y \sim Poi(\mu_i)$, where $E[Y_i] = \mu_i = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} = x_i^T \beta \  \Rightarrow \ q(\mu_i) = \mu_i =  x_i^T \beta  = \eta_i$
* $X_{i1} \sim N(50,10)$
* $X_{i2} \sim U(10,60)$
* $X_{i3} \sim Ber(0.45)$
* $n = 40$
2. Compute $\hat{\mu_i}$  for saturated, null,"full","best" models.
3. Compute Deviance, Rao, Wald statistics for your model and compare final model with the saturated and "full" ones.
4. Generate 100x data for  $n \in \{20,40,60,80,100 \}$ and plot $(\hat{\beta_i} -\beta_i)$ vs. $(n)$
